# COVID-19: United States Confirmed Cases & Deaths 

The following data cleaning and merging is performed to load the data into Power BI for further visualization and analysis.

In [1]:
# Packages / libraries
import os 
import numpy as np 
import pandas as pd
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

The data collected for this analysis is operated by the Johns Hopkins University Center for Systems Science and Engineering (https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)

In [2]:
# load raw data from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
confirmed = pd.read_csv("C:/Users/garcr/Desktop/time_series_covid19_confirmed_US.csv")
deaths = pd.read_csv("C:/Users/garcr/Desktop/time_series_covid19_deaths_US.csv")

# Confirm that the datasets have the same number of rows (countries) and columns (dates)
# Deaths dataframe has an extra column 'Population'
print('The shape of confirmed is:', confirmed.shape)
print('The shape of deaths is:', deaths.shape)

confirmed.head()

The shape of confirmed is: (3262, 104)
The shape of deaths is: (3262, 105)


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,133,135,135,136,136,136,136,136,136,139
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,11,13,13,13,14,14,14,14,14,14
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,923,974,1043,1068,1118,1213,1252,1298,1252,1416
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,51,51,51,51,53,53,53,53,54,54


In [3]:
# Drop unnecessary columns
confirmed.drop(['iso2','iso3','code3','FIPS','Combined_Key'], axis=1, inplace=True)
deaths.drop(['iso2','iso3','code3','FIPS','Combined_Key'], axis=1, inplace=True)

#confirmed.head()
#deaths.head()

In [4]:
# melt the date columns into one column
confirmed2 = pd.melt(confirmed, id_vars= ['UID','Admin2','Province_State','Country_Region','Lat','Long_'], var_name=['Date'])
deaths2 = pd.melt(deaths, id_vars= ['UID','Admin2','Province_State','Country_Region','Lat','Long_','Population'], var_name=['Date'])

print('The shape of confirmed is:', confirmed2.shape)
print('The shape of deaths is:', deaths2.shape)

confirmed2.head(10)

The shape of confirmed is: (303366, 8)
The shape of deaths is: (303366, 9)


,UID,Admin2,Province_State,Country_Region,Lat,Long_,Date,value
0,16.0,NaN,American Samoa,US,-14.271000,-170.132000,1/22/20,0
1,316.0,NaN,Guam,US,13.444300,144.793700,1/22/20,0
2,580.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,1/22/20,0
3,630.0,NaN,Puerto Rico,US,18.220800,-66.590100,1/22/20,0
4,850.0,NaN,Virgin Islands,US,18.335800,-64.896300,1/22/20,0
5,84001001.0,Autauga,Alabama,US,32.539527,-86.644082,1/22/20,0
6,84001003.0,Baldwin,Alabama,US,30.727750,-87.722071,1/22/20,0
7,84001005.0,Barbour,Alabama,US,31.868263,-85.387129,1/22/20,0
8,84001007.0,Bibb,Alabama,US,32.996421,-87.125115,1/22/20,0
9,84001009.0,Blount,Alabama,US,33.982109,-86.567906,1/22/20,0


In [5]:
# convert Date column into datetime objects (xxxx-xx-xx)
confirmed2['Date'] = pd.to_datetime(confirmed2['Date'])
deaths2['Date'] = pd.to_datetime(deaths2['Date'])

#confirmed2.head()

In [6]:
# Replace column headings with detailed fields
confirmed2.columns = confirmed2.columns.str.replace('value','Confirmed')
confirmed2.columns = confirmed2.columns.str.replace('Admin2','County')
deaths2.columns = deaths2.columns.str.replace('value', 'Deaths')
deaths2.columns = deaths2.columns.str.replace('Admin2','County')

# Investigate NULLs before join
print(confirmed2.isnull().sum())
print(deaths2.isnull().sum())

UID                93
County            651
Province_State      0
Country_Region      0
Lat                93
Long_              93
Date                0
Confirmed           0
dtype: int64
UID                93
County            651
Province_State      0
Country_Region      0
Lat                93
Long_              93
Population          0
Date                0
Deaths              0
dtype: int64


The null county values are a result of the District of Columbia, and the 5 inhabited US territories (American Samoa, Guam, Northern Mariana Islands, Puerto Rico, US Virgin Islands) not having counties, in addition to the Grand Princess and Diamond Princess cruise ships.  

In [7]:
covid = confirmed2.merge(deaths2[['UID','Province_State','Country_Region','Date','Population','Deaths']],
                      how='outer',
                      left_on=['UID','Province_State','Country_Region','Date'],
                      right_on=['UID','Province_State','Country_Region','Date'])

# Investigate Nulls post join
print(covid.isnull().sum())
print('\nThe shape of the joined dataframe is:', covid.shape)
#covid.tail()

UID                93
County            651
Province_State      0
Country_Region      0
Lat                93
Long_              93
Date                0
Confirmed           0
Population          0
Deaths              0
dtype: int64

The shape of the joined dataframe is: (303366, 10)


In [8]:
# Fill County NaN values with Province_State values (Applies only to DC, US territories, and cruise ships)
covid['County'].fillna(covid['Province_State'],inplace=True)

#covid.head()
covid.isnull().sum()

UID               93
County             0
Province_State     0
Country_Region     0
Lat               93
Long_             93
Date               0
Confirmed          0
Population         0
Deaths             0
dtype: int64

In [9]:
# Add Month-Year column
covid['Month-Year'] = covid['Date'].dt.strftime('%b-%Y')
covid.tail()

,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Population,Deaths,Month-Year
303361,84070017.0,Southeast Utah,Utah,US,38.996171,-110.701396,2020-04-23,7,0,0,Apr-2020
303362,84070018.0,Southwest Utah,Utah,US,37.854472,-111.441876,2020-04-23,76,0,1,Apr-2020
303363,84070019.0,TriCounty,Utah,US,40.124915,-109.517442,2020-04-23,9,0,0,Apr-2020
303364,84070020.0,Weber-Morgan,Utah,US,41.271160,-111.914512,2020-04-23,136,0,2,Apr-2020
303365,NaN,Southwest,Utah,US,NaN,NaN,2020-04-23,0,0,0,Apr-2020


In [10]:
# Copy df
temp = covid.copy()

# Create columns for previous date's COVID-19 cases to create daily numbers and the running total 
temp['Temp Date'] = covid['Date'] + pd.Timedelta(days=1)
temp.rename(columns={'Confirmed':'Confirmed - 1', 'Deaths':'Deaths - 1', 'Date':'Date - 1'}, inplace=True)

# Perform left join on DFs
covid_19 = covid.merge(temp[['UID','Province_State','Country_Region','Confirmed - 1','Deaths - 1',
                             'Temp Date','Date - 1']], how='left', 
                       left_on=['UID','Province_State','Country_Region','Date'],
                       right_on=['UID','Province_State','Country_Region','Temp Date'])
print(covid_19.shape)
covid_19.head()

(303366, 15)


,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Population,Deaths,Month-Year,Confirmed - 1,Deaths - 1,Temp Date,Date - 1
0,16.0,American Samoa,American Samoa,US,-14.2710,-170.1320,2020-01-22,0,55641,0,Jan-2020,NaN,NaN,NaT,NaT
1,316.0,Guam,Guam,US,13.4443,144.7937,2020-01-22,0,164229,0,Jan-2020,NaN,NaN,NaT,NaT
2,580.0,Northern Mariana Islands,Northern Mariana Islands,US,15.0979,145.6739,2020-01-22,0,55144,0,Jan-2020,NaN,NaN,NaT,NaT
3,630.0,Puerto Rico,Puerto Rico,US,18.2208,-66.5901,2020-01-22,0,2933408,0,Jan-2020,NaN,NaN,NaT,NaT
4,850.0,Virgin Islands,Virgin Islands,US,18.3358,-64.8963,2020-01-22,0,107268,0,Jan-2020,NaN,NaN,NaT,NaT


In [11]:
# Calculate the daily numbers for confirmed cases & deaths (current aggregate - previous aggregate = daily confirmed)
covid_19['Daily Confirmed'] = covid_19['Confirmed'] - covid_19['Confirmed - 1']
covid_19['Daily Deaths'] = covid_19['Deaths'] - covid_19['Deaths - 1']

print(covid_19.shape)
covid_19.head()

(303366, 17)


,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Population,Deaths,Month-Year,Confirmed - 1,Deaths - 1,Temp Date,Date - 1,Daily Confirmed,Daily Deaths
0,16.0,American Samoa,American Samoa,US,-14.2710,-170.1320,2020-01-22,0,55641,0,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
1,316.0,Guam,Guam,US,13.4443,144.7937,2020-01-22,0,164229,0,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
2,580.0,Northern Mariana Islands,Northern Mariana Islands,US,15.0979,145.6739,2020-01-22,0,55144,0,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
3,630.0,Puerto Rico,Puerto Rico,US,18.2208,-66.5901,2020-01-22,0,2933408,0,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
4,850.0,Virgin Islands,Virgin Islands,US,18.3358,-64.8963,2020-01-22,0,107268,0,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN


In [12]:
# Include daily numbers for the first day of data where there is no previous date available (2020-01-22)
covid_19['Daily Confirmed'].loc[covid_19['Date'] == '2020-01-22'] = covid_19['Confirmed']
covid_19['Daily Deaths'].loc[covid_19['Date'] == '2020-01-22'] = covid_19['Deaths']

# Delete unnecessary columns
del covid_19['Confirmed - 1'] 
del covid_19['Deaths - 1']
del covid_19['Temp Date']
del covid_19['Date - 1']

covid_19.head(5)

,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Population,Deaths,Month-Year,Daily Confirmed,Daily Deaths
0,16.0,American Samoa,American Samoa,US,-14.2710,-170.1320,2020-01-22,0,55641,0,Jan-2020,0.0,0.0
1,316.0,Guam,Guam,US,13.4443,144.7937,2020-01-22,0,164229,0,Jan-2020,0.0,0.0
2,580.0,Northern Mariana Islands,Northern Mariana Islands,US,15.0979,145.6739,2020-01-22,0,55144,0,Jan-2020,0.0,0.0
3,630.0,Puerto Rico,Puerto Rico,US,18.2208,-66.5901,2020-01-22,0,2933408,0,Jan-2020,0.0,0.0
4,850.0,Virgin Islands,Virgin Islands,US,18.3358,-64.8963,2020-01-22,0,107268,0,Jan-2020,0.0,0.0


In [13]:
# checking most recent US daily totals
daily_sum = covid_19.groupby('Date').agg({'Daily Confirmed': 'sum'})
print(daily_sum.tail())
daily_deaths = covid_19.groupby('Date').agg({'Daily Deaths': 'sum'})
print(daily_deaths.tail())

            Daily Confirmed
Date                       
2020-04-19          26612.0
2020-04-20          25517.0
2020-04-21          27539.0
2020-04-22          28355.0
2020-04-23          28950.0
            Daily Deaths
Date                    
2020-04-19        1997.0
2020-04-20        1433.0
2020-04-21        2350.0
2020-04-22        2178.0
2020-04-23        3332.0


In [14]:
# Export Data as csv to load into Power BI
covid_19.to_csv('UScovid', sep='\t')